In [1]:
%load_ext autoreload
%autoreload 2

import cfr
import numpy as np

In [2]:
# create a reconstruction job object using the `cfr.ReconJob` class
job = cfr.ReconJob()

# load the pseudoPAGES2k database from a netCDF file
job.proxydb = cfr.ProxyDatabase().load_nc('data/ppe/ppwn_SNR10_rta.nc')

job.load_clim(
    tag='prior',
    path_dict={
        'tas': 'data/ppe/tas_sfc_Amon_iCESM_past1000historical_085001-200512.nc',
        'pr': 'data/ppe/',
    },
    anom_period=(1850, 1870),  # calculate anomalies against the reference interval 1850-1870 CE
    verbose=True,
)

job.load_clim(
    tag='obs',
    path_dict={
        'tas': 'data/ppe/tas_sfc_Amon_iCESM_past1000historical_085001-200512.nc',
        'pr': 'data/ppe/',
    },
    anom_period=(1850, 1870),
    verbose=True,
)

# PSM to be used
ptype_psm_dict = {
    'tree.TRW': 'Bilinear',
    'tree.MXD': 'Linear',
    'coral.d18O': 'Linear',
    'coral.SrCa': 'Linear',
    'ice.d18O': 'Linear',
    'lake.varve_thickness': 'Linear',
}

# Seasonality for each proxy type
ptype_season_dict = {
    'tree.TRW': [  # expert curated pool of possible growing seasons
        [1,2,3,4,5,6,7,8,9,10,11,12],
        [6,7,8],
        [3,4,5,6,7,8],
        [6,7,8,9,10,11],
        [-12,1,2],
        [-9,-10,-11,-12,1,2],
        [-12,1,2,3,4,5],
    ],
    'tree.MXD': [  # expert curated pool of possible growing seasons
        [1,2,3,4,5,6,7,8,9,10,11,12],
        [6,7,8],
        [3,4,5,6,7,8],
        [6,7,8,9,10,11],
        [-12,1,2],
        [-9,-10,-11,-12,1,2],
        [-12,1,2,3,4,5],
    ],
    'coral.d18O': list(range(1, 13)),            # annual
    'coral.SrCa': list(range(1, 13)),            # annual
    'ice.d18O': list(range(1, 13)),              # annual
    'lake.varve_thickness': list(range(1, 13)),  # annual
}

job.calib_psms(
    ptype_psm_dict=ptype_psm_dict,
    ptype_season_dict=ptype_season_dict,
    verbose=True,)

job.forward_psms(verbose=True)

job.annualize_clim(tag='prior', verbose=True, months=list(range(1, 13)))

job.regrid_clim(tag='prior', nlat=42, nlon=63, verbose=True)

job.save('data/ppe/job-lmr-ppe-pages2k', verbose=True)

job = cfr.ReconJob()
job.load('data/ppe/job-lmr-ppe-pages2k/', verbose=True)

job.prior['pr'].da.values *= 1e5  # to regularize the pr values

job.run_da_mc(
    save_dirpath='data/ppe/results/lmr-ppe-pages2k',
    recon_seeds=list(range(1, 21)),
    recon_vars=['tas', 'pr'],  # to reconstruct the tas and pr fields
    recon_period=[800, 2000],
    verbose=True,
)

>>> job.configs["prior_path"] = {'tas': 'data/ppe/tas_sfc_Amon_iCESM_past1000historical_085001-200512.nc', 'pr': 'data/ppe/'}
>>> job.configs["prior_anom_period"] = (1850, 1870)
>>> job.configs["prior_lon_name"] = lon
>>> job.configs["prior_time_name"] = time


/work/croppers/miniconda3/envs/LMRt/lib/python3.8/site-packages/xarray/backends/plugins.py:139: RuntimeWarning: 'netcdf4' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)
/work/croppers/miniconda3/envs/LMRt/lib/python3.8/site-packages/xarray/backends/plugins.py:139: RuntimeWarning: 'scipy' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)
/work/croppers/miniconda3/envs/LMRt/lib/python3.8/site-packages/xarray/backends/plugins.py:148: RuntimeWarning: 'netcdf4' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)
/work/croppers/miniconda3/envs/LMRt/lib/python3.8/site-packages/xarray/backends/plugins.py:148: RuntimeWarning: 'h5netcdf' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)
/work/croppers/miniconda3/envs/LMRt/lib/python3.8/site-packages/xarray/backends/plugins.py:148: RuntimeWarning: 'scipy' fails while guessing
  warnings.warn(

ValueError: did not find a match in any of xarray's currently installed IO backends ['netcdf4', 'scipy']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html
https://docs.xarray.dev/en/stable/user-guide/io.html

In [ ]:
import xarray as xr
xr.open_dataset('data/ppe/tas_sfc_Amon_iCESM_past1000historical_085001-200512.nc')